# Building a protein-protein interection network

In [15]:
import graph_tool_extras as gte

## Introduction

In this notebook, a network of protein-protein interactions in human bodies was built.
The database in the file below is used. It's from the Molecular Interaction Searching Tool's dataset at https://fgrtools.hms.harvard.edu/MIST.

In [16]:
PATH = 'Datasets/MIST_interaction_ppi_vs5_0-9606.txt'

## Understanding the data

MIST_interaction_ppi_vs5_0-9606.txt has 18 columns and 1048431 rows, each row representing a protein-protein interaction. To understand each column, check the documentation below extracted from https://fgrtools.hms.harvard.edu/MIST/downloads.jsp.
1. MasterNetID: our own unique interaction ID
2. TaxID_A: taxonomy id for interacting partner A, for example, 9606 for human genes
3. GeneA: entrez geneid for interacting partner A
4. TaxID_B: taxonomy id for interacting partner B, for example, 9606 for human genes
5. GeneB: entrez geneid for interacting partner B
6. Rank: confidence of interaction (high, moderate and low)
7. Interaction_type: PPI, genetic, interolog, interolog-genetic
8. Exp_Direct: evidence codes (MI ID) for direct interaction eg. "MI:0018" for 2-Hybrid interaction
9. Exp_Indirect: evidence code for in-direct interaction eg. "MI:0488" for psi-mi
10. TaxID_interolog: this column is only populated for interolog and shows the taxonomy ID of the original data
11. Source_databases: eg. BioGrid
12. Reference: Pubmed_ID or other reference ID (eg reference from FlyBase)
13. Reference_type: eg. "PMID"
14. Count_direct: count of unique evidence code (MI number) as direct interaction
15. Count_indirect: count of unique evidence code (MI number) as in-direct interaction
16. Count_paper: count of unique reference IDs
17. Source_Interolog: this column is only populated for interolog and shows the MasterNetID of the original data
18. Comment

In [30]:
def get_or_add_vertex(g, id):
    u = g.vertex_by_id(id)
    if u is None:
        u = g.add_vertex_by_id(id)
       # u['id'] = 0
    return u

def get_or_add_edge(g, gene_a, gene_b, master_net_id):
    e = g.edge_by_ids(gene_a, gene_b)
    if e is None:
        e = g.add_edge_by_ids(gene_a, gene_b)
        #e['interaction_id'] = master_net_id
    return e

## Reading the data and building the network

In [31]:
g = gte.Graph(directed=False)

with open('mist_ppi_human.txt') as file:

    # Ignoring the first line because it's the header.
    next(file)
    i = 0
    # Reading the file line by line, without fully loading it into the memory.
    for line in file:
        
        # Turns the line into a list of parts
        # considering '\t' as a separator.
        parts = line.split('\t')

        # Ignoring the first and the last character
        # of each part to eliminate quotation marks.
        parts = [part[1:-1] for part in parts]

        rank = parts[5]
        # Conceitos dos Vertices: Um vertice representará um Gene
        # Conceito das arestas: Uma aresta entre GeneA e GeneB indica que há interação mediana ou forte entre ambas proteínas
        # Operacionalização dos vértices: Cada vertice corresponde a um Gene, independente de ser A ou B, ou de existirem conexões.
        # Operacionalização das arestas: Apenas existem arestas caso um par de Genes têm interação proteíca entre si.
        
        # if the rank is high, there is interaction between GeneA and GeneB
        gene_a = parts[2]
        gene_b = parts[4]
        interaction_id = parts[0]

        vertex_ga = get_or_add_vertex(g, gene_a)
        vertex_gb = get_or_add_vertex(g, gene_b)
        if rank == 'high':
            e = get_or_add_edge(g, gene_a, gene_b, interaction_id)
            
            i += 1
            
        
        # g = gte.Graph(directed=False)
    print(i)

499969


In [ ]:
gte.clean(g)

In [ ]:
from graph_tool import draw
import netpixi

In [ ]:
layout = draw.sfdp_layout(g)

gte.move(g, layout)

In [ ]:
gte.save(g, 'misthumanv2.net.gz')

In [ ]:
r = netpixi.render('misthumanv2.net.gz')